In [14]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Ridge
from scipy.sparse import hstack

In [2]:
# 1. Загрузите данные об описаниях вакансий и соответствующих годовых зарплатах из файла salary-train.csv.

In [42]:
data = pd.read_csv('salary-train.csv')

In [1]:
data_train = pd.read_csv('salary-train.csv')

In [40]:
data_test = pd.read_csv('salary-test-mini.csv')

In [7]:
data.head(5)

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


In [6]:
# 2. Проведите предобработку:
# - Приведите тексты к нижнему регистру (text.lower())

In [48]:
data_train.apply(lambda x: x.astype(str).str.lower())

In [49]:
data_train['FullDescription'][:10]

0    international sales manager london ****k  ****...
1    an ideal opportunity for an individual that ha...
2    online content and brand manager// luxury reta...
3    a great local marketleader is seeking a perman...
4    registered nurse / rgn  nursing home for young...
5    sales and marketing assistant will provide adm...
6    vacancy ladieswear fashion area manager / regi...
7    reference: lr/jan/**** our client is one of th...
8    sponsorship manager london the company a marke...
9    about barclays barclays moves, lends, invests ...
Name: FullDescription, dtype: object

In [23]:
# Замените все, кроме букв и цифр, на пробелы — это облегчит дальнейшее разделение текста на слова. 
# Для такой замены в строке text подходит следующий вызов: re.sub('[^a-zA-Z0-9]', ' ', text). 
# Также можно воспользоваться методом replace у DataFrame, чтобы сразу преобразовать все тексты:

In [50]:
data_train['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex=True)

In [51]:
data_train['FullDescription'][:10]

0    international sales manager london     k      ...
1    an ideal opportunity for an individual that ha...
2    online content and brand manager   luxury reta...
3    a great local marketleader is seeking a perman...
4    registered nurse   rgn  nursing home for young...
5    sales and marketing assistant will provide adm...
6    vacancy ladieswear fashion area manager   regi...
7    reference  lr jan      our client is one of th...
8    sponsorship manager london the company a marke...
9    about barclays barclays moves  lends  invests ...
Name: FullDescription, dtype: object

In [39]:
# Примените TfidfVectorizer для преобразования текстов в векторы признаков. 
# Оставьте только те слова, которые встречаются хотя бы в 5 объектах (параметр min_df у TfidfVectorizer).

In [68]:
vec = TfidfVectorizer(min_df=5)
x_train_transform = vec.fit_transform(data_train['FullDescription'])

In [52]:
# Замените пропуски в столбцах LocationNormalized и ContractTime на специальную строку 'nan'. 

In [64]:
data_train['LocationNormalized'].fillna('nan', inplace=True)
data_train['ContractTime'].fillna('nan', inplace=True)

In [66]:
data_train.head(10)

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london k ...,london,permanent,33000
1,an ideal opportunity for an individual that ha...,london,permanent,50000
2,online content and brand manager luxury reta...,south east london,permanent,40000
3,a great local marketleader is seeking a perman...,dereham,permanent,22500
4,registered nurse rgn nursing home for young...,sutton coldfield,nan,20355
5,sales and marketing assistant will provide adm...,crawley,nan,22500
6,vacancy ladieswear fashion area manager regi...,uk,permanent,32000
7,reference lr jan our client is one of th...,bristol,permanent,30000
8,sponsorship manager london the company a marke...,central london,permanent,31500
9,about barclays barclays moves lends invests ...,south east london,permanent,42499


In [67]:
# Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime.

In [71]:
enc = DictVectorizer()
x_train_categ = enc.fit_transform(data_train[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [73]:
# Объедините все полученные признаки в одну матрицу "объекты-признаки". 
# Обратите внимание, что матрицы для текстов и категориальных признаков являются разреженными. 
# Для объединения их столбцов нужно воспользоваться функцией scipy.sparse.hstack.

In [75]:
x_train = hstack([x_train_transform, x_train_categ])

In [81]:
y_train = data_train['SalaryNormalized']

In [78]:
# 3. Обучите гребневую регрессию с параметрами alpha=1 и random_state=241. 
# Целевая переменная записана в столбце SalaryNormalized.

In [82]:
model = Ridge(alpha=1,random_state=241)
model.fit(x_train, y_train)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=241, solver='auto', tol=0.001)

In [83]:
# 4. Постройте прогнозы для двух примеров из файла salary-test-mini.csv. 
# Значения полученных прогнозов являются ответом на задание. Укажите их через пробел.

In [84]:
x_test_transform = vec.transform(data_test['FullDescription'])

In [85]:
x_test_categ = enc.transform(data_test[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [86]:
x_test = hstack([x_test_transform, x_test_categ])

In [95]:
y_test = model.predict(x_test)
print('{:0.2f} {:0.2f}'.format(y_test[0],y_test[1]))

55850.59 37182.51
